Here I import the necessary packages

In [3]:
# Import Pennylane packages
import pennylane as qml
import cirq
from pennylane_cirq import ops as cirq_ops
from pennylane import qaoa

# Tensorflow packages
import tensorflow as tf

# Other packages
import numpy as np
import networkx as nx
import random

Definition of the graph I want to use in my problem

In [4]:
# Definition of the graph

def dregular_graph(n,d,mu,sigma):
    G= nx.generators.random_graphs.random_regular_graph(d,n)
    for e in list(G.edges):
        G[e[0]][e[1]]["weight"] = round(random.gauss(mu,sigma),2)
        #G[e[0]][e[1]]["weight"] = 1
    return G

G = dregular_graph(10,8,0.1, 1)

My circuit implementation. I define the `U_C` gate explicitly because I want to take into account the weights provided by my graph and with `qaoa.maxcut(G)` it sends them to one. However, the mixer part of the Hamiltonian is still useful so I employ it.

In [17]:
# Define the qaoa problem
cost_h, mixer_h = qaoa.maxcut(G)

# Definition of the mixer and cost gates, respectively
def qaoa_mixer(alpha):
    qaoa.mixer_layer(alpha, mixer_h)

def U_C(gamma):
    for e in list(G.edges):
        wire1 = int(e[0])
        wire2 = int(e[1])
        qml.CNOT(wires=[wire1, wire2])
        qml.RZ(G[wire1][wire2]["weight"]*gamma, wires=wire2)
        qml.CNOT(wires=[wire1, wire2])

# Define the depth my circuit depth
p = 1

# Definition of the circuit together with the device
dev = qml.device('default.qubit.autograd', wires = len(G.nodes), shots = 5000)
@qml.qnode(dev, diff_method = "backprop")
def circuit(params, **kwargs):
    for i in range(len(G.nodes)):
        qml.Hadamard(wires = i)
    for j in range(p):
        U_C(params[0][j])
        qaoa.mixer_layer(params[1][j], mixer_h)
    return qml.probs(wires = list(range(len(G.nodes))))

The following cell contains some functions that are used to compute the cost function. In principle, the first two should be completely irrelevant (the first one converts an array to a list of integers and the second one computes the MaxCut cost function for a given bitstring). The function that plays the important role here is `cost_function`, which is the one I employ in the optimization part.

In [15]:
# Converts a string into a list of ints

def string_to_list(x):
    list1 = []
    for el in x:
        list1.append(int(el))
    return list1

# MaxCut cost function

def MaxCut_cost(G,string):
    x = string_to_list(string)
    C = 0.
    for edges in list(G.edges):
        i = edges[0]
        j = edges[1]
        C += G[i][j]["weight"]*(x[i] + x[j]-2*x[i]*x[j])
    return C

def cost_function(params):
    counts = {}
    result = circuit(params)
    # In the following line, change 2 --> your number of qubits
    for i in range(len(result)):
        counts[f"{i:010b}"] = result[i]
    E = 0        
    for bitstring in counts.keys():
        x = string_to_list(bitstring)
        energy = MaxCut_cost(G, x)
        E += -energy*counts[bitstring]
    return E

Finally, I implement the optimization with the adaptive gradient descent provided by Pennylane. Now I'm using the `default.qubit.autograd` option so in principle I shouldn't deal with tensorflow, should I? Now I get a different error which I don't know how to solve.

In [16]:
opt = qml.AdagradOptimizer(0.1)
steps = 20
vals = []
params = 0.01*np.random.rand(2, p)
print("I'm starting to optimize... Please wait!")
for i in range(steps):
    params = opt.step(cost_function, params)
    #params = params_to_2pi(params)
    print("Objective after step {:5d}: {: .7f}".format(i + 1, alternative_cost_function(params)))

I'm starting to optimize... Please wait!


TypeError: loop of ufunc does not support argument 0 of type ArrayBox which has no callable exp method

Thank you very much for your help! :)